In [1]:
import numpy as np
import os

In [2]:
labels = np.loadtxt('../ISRUC-Sleep Dataset/Two Session Subjects/Subject 1/1/1_1.txt')[:-30]
np.place(labels, labels==5, [4])

In [3]:
rootdir = 'eeg_images'
onlyfiles = [f for f in os.listdir(rootdir) if os.path.isfile(os.path.join(rootdir, f))]

from scipy import ndimage
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

train_files = []
y_train = labels

i=0

for _file in onlyfiles:
    train_files.append(_file)

In [4]:
from PIL import Image

dataset = np.ndarray(shape=(len(train_files), 288, 432, 3), dtype=np.float32)

i = 0
for _file in train_files:
    img = load_img(rootdir + "/" + _file)
    img.thumbnail((432, 288))
    x = img_to_array(img) 
    x = (x - 128.0) / 128.0
    dataset[i] = x
    i += 1

In [5]:
from sklearn.model_selection import train_test_split

#Splitting 
X_train, X_test, y_train, y_test = train_test_split(dataset, y_train, test_size=0.1, random_state=33)

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.python.keras.layers.normalization import BatchNormalization

np.random.seed(1000)
#Instantiate an empty model
model = Sequential()

# 1st Convolutional Layer
model.add(Conv2D(filters=96, input_shape=(288,432,3), kernel_size=(11,11), strides=(4,4), padding='valid'))
model.add(Activation('relu'))
# Max Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))

# 2nd Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(11,11), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Max Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))

# 3rd Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))

# 4th Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))

# 5th Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Max Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))

# Passing it to a Fully Connected layer
model.add(Flatten())
# 1st Fully Connected Layer
model.add(Dense(4096, input_shape=(224*224*3,)))
model.add(Activation('relu'))
# Add Dropout to prevent overfitting
model.add(Dropout(0.4))

# 2nd Fully Connected Layer
model.add(Dense(4096))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))

# 3rd Fully Connected Layer
model.add(Dense(1000))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))

# Output Layer
model.add(Dense(17))
model.add(Activation('softmax'))

model.summary()

# Compile the model
model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy, optimizer='adam', metrics=["accuracy"])

W0106 13:03:16.731564 139823258576704 deprecation.py:506] From /home/isuru/.local/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 70, 106, 96)       34944     
_________________________________________________________________
activation (Activation)      (None, 70, 106, 96)       0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 35, 53, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 25, 43, 256)       2973952   
_________________________________________________________________
activation_1 (Activation)    (None, 25, 43, 256)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 21, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 10, 19, 384)       8

In [ ]:
history = model.fit(X_train, y_train,
          batch_size=2,
          epochs=8,
          validation_data=(X_test, y_test))

Train on 812 samples, validate on 91 samples
Epoch 1/8
812/812 [==============================] - 346s 426ms/sample - loss: 1.9991 - acc: 0.2303 - val_loss: 1.6394 - val_acc: 0.2527
Epoch 2/8
812/812 [==============================] - 374s 460ms/sample - loss: 1.6268 - acc: 0.2475 - val_loss: 1.6268 - val_acc: 0.2088
Epoch 3/8
812/812 [==============================] - 327s 403ms/sample - loss: 1.6158 - acc: 0.2328 - val_loss: 1.6147 - val_acc: 0.2088
Epoch 4/8
812/812 [==============================] - 350s 431ms/sample - loss: 1.6041 - acc: 0.2315 - val_loss: 1.6008 - val_acc: 0.2527
Epoch 5/8
566/812 [===================>..........] - ETA: 1:38 - loss: 1.6038 - acc: 0.2350